In [1]:
from huggingface_hub import notebook_login
import os
notebook_login()
repo_name='jkhyjkhy/wav2vec2-large-xlsr-sw-ASR'

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
BASE = '/content/drive/MyDrive/ASR_wav2vec_project/preprocessing/processed_data/14.0-delta-2023-06-23'
CSV = f'{BASE}/manifest_sw_14_0_delta.csv'
AUDIO_DIR = f'{BASE}/cleaned_sw_audio_14_0_delta'
# If you're using a pre-configured HuggingFace processor, you can comment out or remove VOCAB
VOCAB = f'{BASE}/vocab.json'

In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv(CSV)
# df['wav_path'] = df['wav_path'].apply(lambda p: os.path.join(AUDIO_DIR, p.split('\\')[-1]))
df["wav_path"] = df["wav_path"].apply(lambda p: os.path.join(AUDIO_DIR, os.path.basename(p)))
print(f"Total samples in dataset: {len(df)}")

Total samples in dataset: 270


In [5]:
from datasets import Dataset, Audio
dataset = Dataset.from_pandas(df[['wav_path', 'transcript']])

In [6]:
import re

def remove_special_characters(batch, column_names='transcript'):
  chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\(\)]'
  batch[column_names] = re.sub(chars_to_ignore_regex, '', batch[column_names]).lower() + " "
  return batch

dataset = dataset.map(remove_special_characters)

print(len(dataset['wav_path']))

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

270


In [9]:
from transformers import AutoProcessor, AutoModelForCTC

processor = AutoProcessor.from_pretrained("jkhyjkhy/wav2vec2-large-xlsr-sw-ASR")
model = AutoModelForCTC.from_pretrained("jkhyjkhy/wav2vec2-large-xlsr-sw-ASR")

preprocessor_config.json:   0%|          | 0.00/257 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json:   0%|          | 0.00/598 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [11]:
import torch
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"]).unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  print(pred_ids)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)

  return batch

In [12]:
dataset = dataset.cast_column("wav_path", Audio(sampling_rate=16000))

def prepare_dataset(batch):
    audio = batch["wav_path"]

    audio_inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"])

    batch["input_values"] = audio_inputs["input_values"][0]
    batch["input_length"] = len(batch["input_values"])
    label_inputs = processor.tokenizer(batch["transcript"])
    batch["labels"] = label_inputs["input_ids"]

    return batch

In [17]:
dataset.column_names
embeded_dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

AttributeError: 'Dataset' object has no attribute 'keys'

In [21]:
embeded_dataset.column_names

['input_values', 'input_length', 'labels']

In [24]:
!pip install evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.6 MB/s eta 0:00:00


In [25]:
import evaluate
wer_metric = evaluate.load('wer')

In [26]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [27]:
results = embeded_dataset.map(map_to_result, remove_columns=embeded_dataset.column_names)
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Parameter 'function'=<function map_to_result at 0x7c52257f7a60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/270 [00:00<?, ? examples/s]

tensor([[51, 51, 51, 51, 51, 51, 51, 12, 51,  2, 51, 51, 51, 51, 51, 14, 15, 51,
         35, 48, 48, 51, 31, 31,  2, 51, 51, 51, 51, 51, 51, 51, 51, 40, 38, 51,
         48, 51, 51, 51, 51, 51, 51, 51, 51, 14, 35, 35, 48, 51, 51, 31, 51,  2,
         48, 51, 51, 51, 51, 51, 14, 51, 51, 51, 43, 51,  2, 51, 51, 51, 51, 51,
         51, 51, 51, 51, 14, 51, 33, 51, 48, 48, 51, 51, 51, 51, 51, 51, 51, 40,
         21, 21,  2, 48, 48, 51, 51, 19, 51, 20, 51, 51, 33, 51, 51, 14, 14, 38,
         51, 51, 51, 51, 51, 51, 40, 51,  2, 51, 51, 51, 51, 51, 51, 51, 14, 51,
         33, 48, 48, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 21,  2, 51, 48, 51,
         51, 51, 14, 51, 37, 51,  2, 51, 51, 51, 51, 51, 51, 51, 14, 35, 51, 48]])
tensor([[51, 51, 51, 51, 51,  6, 51, 51,  2, 51, 51, 51, 51, 37, 51, 35, 51, 51,
         51, 51, 51, 36, 51, 51, 35, 48, 48, 51, 51, 51, 51, 31, 51,  2, 48, 48,
         51, 21,  2, 51, 33, 33, 51, 51, 51, 51, 51, 14, 51, 15, 51, 33, 51, 51,
         51, 51, 51, 51, 5

In [28]:
from IPython.display import display, HTML
import random
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [29]:
show_random_elements(results)

,pred_str,text
0,hivi lee zinaendelea kutoa nafasi kisalama kuwasaidia watoto wapike na wanawake kujujana,hivi leo kinaendelea kutoa nafasi salama kuwasaidia watoto wa kike na wanawake vijana
1,mifugo inaweza kuathiriwa na ugonjwa huu wa upele,mifugo inaweza kuathiriwa na ugonjwa huu wa upele
2,weka safuria yako jikoni,weka sufuria yako jikoni
3,kanda unga na ti mafuta kiasi,kanda unga na tia mafuta kiasi
4,mnyama asiye mgonjwa badaa anapoonyawadamu wingine,mnyama asiye mgonjwa baadaye anapomnyonya damu mnyama mwingine
5,alisema jambo muhimu ni kuchukua msima modhabiti na kuzuia manyanyaso dhidi ya raia,alisema jambo muhimu ni kuchukua msimamo thabiti na kuzuia manyanyaso dhidi ya raia
6,katika siku ya pili ya vikao vya seneti kuhusu uteuzi wake katika maakama ya ju jaji katanji braun jakson alitabiliwa na maswali kwa saa kadha,katika siku ya pili ya vikao vya seneti kuhusu uteuzi wake katika mahakama ya juu jaji ketanji brown jackson alikabiliwa na maswali kwa saa kadhaa
7,ugonjwa wa miguu na midomo,ugonjwa wa miguu na midomo
8,huwa kwa nadra au mara chache la kisababishia nimbako,huua kwa nadra au mara chache lakini husababisha mimba kutoka
9,matumizi ya viazilisha,matumizi ya viazi lishe
